In [1]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

## Read Data
You can change the code below to read your own data.

In [2]:
df = pd.read_csv(filepath_or_buffer="./input/heart.csv", header=0, index_col=None)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## Train a classification model
In this step, you can either train a model in the cell below, or skip the training part and read the model prediction of the training data from your own model .

In [4]:
'''prepare data'''
X = df.drop(columns=['target']).values
y = df['target'].values
y = y.reshape(len(y))

train, test, train_labels, test_labels = train_test_split(X, y, test_size = 0.2)

train_df = pd.DataFrame(train, columns=df.columns[:-1].values)
test_df = pd.DataFrame(test, columns=df.columns[:-1].values)

'''train the model'''
clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(train, train_labels)

'''report accuracy'''
print(clf.score(test,test_labels ))

0.819672131147541


In [5]:
''' obtain the original model's prediction '''
y_pred = clf.predict(train)

## Output Training Data

In [6]:
data_name = 'heart'

'''
cols: the column names of the input data.
data: the training input.
target_names: the names for the target classes (grount truth).
real_min: minimal values for columns of the input data.
real_max: maximal values for columns of the input data.
y_pred: the prediction of the input data from the model you want to explain.
y_gt: the ground truth of the input data.
'''

def output_data(cols, data, target_names, real_min, real_max, y_pred, y_gt):
    filename = "./output/"+data_name+"/test.json"
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    to_output = {}
    to_output['columns'] = cols
    to_output['data'] = data
    to_output['target_names'] = target_names
    to_output['real_min'] = real_min
    to_output['real_max'] = real_max
    to_output['y_pred'] = y_pred
    to_output['y_gt'] = y_gt
    with open(filename, 'w') as output:
        output.write(json.dumps(to_output))

In [8]:
''' name the target classes '''
target_names = ["False", "True"]

''' obtain the min and max values '''
min_val = np.min(X, axis=0)
max_val = np.max(X, axis=0)

''' output data '''
output_data(cols=df.drop(columns=['target']).columns.values.tolist(), 
            data=train.tolist(), 
            target_names=target_names, 
            real_min=min_val.tolist(), 
            real_max=max_val.tolist(),
            y_pred=y_pred.tolist(), 
            y_gt=train_labels.tolist())